# Jobs API Setup
## Purpose: Create test jobs and validate Jobs API functionality

This notebook creates sample jobs and tests Jobs API functionality.
The original Jobs API testing showed partial success (60%) with some endpoints returning 404.

**Prerequisites:**
- Valid VeevaVault authentication
- Job management permissions
- Background job capabilities

In [1]:
# Import required libraries
import sys
import os
import time

# Add the parent directory to sys.path to import our modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from veevavault.client.vault_client import VaultClient
from veevavault.services.authentication import AuthenticationService
from test_credentials import TEST_VAULTS, DEFAULT_VAULT

# Get vault configuration
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_URL = vault_config["URL"]
VAULT_USERNAME = vault_config["username"]
VAULT_PASSWORD = vault_config["password"]

print("📦 Libraries imported successfully")
print(f"🔗 Target Vault: {VAULT_URL}")
print(f"👤 Username: {VAULT_USERNAME}")

📦 Libraries imported successfully
🔗 Target Vault: https://vv-consulting-michael-mastermind.veevavault.com/
👤 Username: veevatools@vv-consulting.com


In [2]:
# Initialize client and authenticate
print("🔐 Initializing VeevaVault client...")
client = VaultClient()
client.vaultURL = VAULT_URL.rstrip('/')

print("🔑 Authenticating...")
auth_service = AuthenticationService(client)
VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")

auth_response = auth_service.authenticate_with_username_password(
    username=VAULT_USERNAME,
    password=VAULT_PASSWORD,
    vaultDNS=VAULT_DNS
)

if auth_response.get('responseStatus') == 'SUCCESS':
    session_id = auth_response['sessionId']
    vault_id = auth_response.get('vaultId', 'Unknown')
    print(f"✅ Authentication successful!")
    print(f"🆔 Session ID: {session_id[:20]}...")
    print(f"🏢 Vault ID: {vault_id}")
    auth_success = True
else:
    print(f"❌ Authentication failed: {auth_response}")
    auth_success = False

🔐 Initializing VeevaVault client...
🔑 Authenticating...
✅ Authentication successful!
🆔 Session ID: 2C5F99B371380857552D...
🏢 Vault ID: 92425
✅ Authentication successful!
🆔 Session ID: 2C5F99B371380857552D...
🏢 Vault ID: 92425


In [3]:
# Test: Check job histories (this worked in original test)
if auth_success:
    print("📋 Testing job histories retrieval...")
    
    histories_response = client.api_call("api/v25.2/services/jobs/histories")
    
    if histories_response.get('responseStatus') == 'SUCCESS':
        jobs_data = histories_response.get('jobs', [])
        total_jobs = histories_response.get('responseDetails', {}).get('total', 0)
        print(f"✅ Found {len(jobs_data)} jobs in response (Total: {total_jobs})")
        
        if jobs_data:
            print("\n📊 Recent job examples:")
            for i, job in enumerate(jobs_data[:3]):  # Show first 3
                job_id = job.get('job_id', 'Unknown')
                job_title = job.get('title', 'Unknown')
                job_status = job.get('status', 'Unknown')
                print(f"  {i+1}. Job {job_id}: {job_title} ({job_status})")
            
            # Save a job ID for testing
            test_job_id = jobs_data[0].get('job_id')
            print(f"\n🎯 Selected job for testing: {test_job_id}")
    else:
        print(f"❌ Failed to get job histories: {histories_response}")
        test_job_id = None

📋 Testing job histories retrieval...
✅ Found 50 jobs in response (Total: 3969)

📊 Recent job examples:
  1. Job 264474: Task Reminder Notification (SUCCESS)
  2. Job 264461: Clean up temp file handle metadata (SUCCESS)
  3. Job 264457: Cleanup User Exceptions (SUCCESS)

🎯 Selected job for testing: 264474
✅ Found 50 jobs in response (Total: 3969)

📊 Recent job examples:
  1. Job 264474: Task Reminder Notification (SUCCESS)
  2. Job 264461: Clean up temp file handle metadata (SUCCESS)
  3. Job 264457: Cleanup User Exceptions (SUCCESS)

🎯 Selected job for testing: 264474


In [4]:
# Test: Check job monitors (this also worked)
if auth_success:
    print("🔍 Testing job monitors...")
    
    monitors_response = client.api_call("api/v25.2/services/jobs/monitors")
    
    if monitors_response.get('responseStatus') == 'SUCCESS':
        monitors_data = monitors_response.get('jobs', [])
        total_monitors = monitors_response.get('responseDetails', {}).get('total', 0)
        print(f"✅ Found {len(monitors_data)} active/scheduled jobs (Total: {total_monitors})")
        
        if monitors_data:
            print("\n📊 Active job monitors:")
            for i, job in enumerate(monitors_data[:3]):  # Show first 3
                job_id = job.get('job_id', 'Unknown')
                job_title = job.get('title', 'Unknown')
                job_status = job.get('status', 'Unknown')
                print(f"  {i+1}. Job {job_id}: {job_title} ({job_status})")
            
            # Save a scheduled job ID if available
            scheduled_jobs = [job for job in monitors_data if job.get('status') == 'SCHEDULED']
            if scheduled_jobs:
                scheduled_job_id = scheduled_jobs[0].get('job_id')
                print(f"\n🎯 Found scheduled job for testing: {scheduled_job_id}")
            else:
                scheduled_job_id = None
                print("\n⚠️ No scheduled jobs found")
    else:
        print(f"❌ Failed to get job monitors: {monitors_response}")
        scheduled_job_id = None

🔍 Testing job monitors...
✅ Found 4 active/scheduled jobs (Total: 4)

📊 Active job monitors:
  1. Job 264561: Cleanup User Exceptions (SCHEDULED)
  2. Job 264562: User Account Activation (SCHEDULED)
  3. Job 264568: Clean up temp file handle metadata (SCHEDULED)

🎯 Found scheduled job for testing: 264561
✅ Found 4 active/scheduled jobs (Total: 4)

📊 Active job monitors:
  1. Job 264561: Cleanup User Exceptions (SCHEDULED)
  2. Job 264562: User Account Activation (SCHEDULED)
  3. Job 264568: Clean up temp file handle metadata (SCHEDULED)

🎯 Found scheduled job for testing: 264561


In [5]:
# Test: Individual job status (this worked in original test)
if auth_success and test_job_id:
    print(f"🔍 Testing individual job status for job {test_job_id}...")
    
    job_response = client.api_call(f"api/v25.2/services/jobs/{test_job_id}")
    
    if job_response.get('responseStatus') == 'SUCCESS':
        job_data = job_response.get('data', {})
        print(f"✅ Successfully retrieved job details")
        print(f"  📊 Job ID: {job_data.get('id', 'Unknown')}")
        print(f"  📊 Status: {job_data.get('status', 'Unknown')}")
        print(f"  📊 Created: {job_data.get('created_date', 'Unknown')}")
        print(f"  📊 Method: {job_data.get('method', 'Unknown')}")
    else:
        print(f"❌ Failed to get job details: {job_response}")

🔍 Testing individual job status for job 264474...
✅ Successfully retrieved job details
  📊 Job ID: 264474
  📊 Status: SUCCESS
  📊 Created: 2025-08-30T09:00:35.000Z
  📊 Method: Unknown
✅ Successfully retrieved job details
  📊 Job ID: 264474
  📊 Status: SUCCESS
  📊 Created: 2025-08-30T09:00:35.000Z
  📊 Method: Unknown


In [6]:
# Test: Start job endpoint (this was the issue in original test)
if auth_success and scheduled_job_id:
    print(f"🚀 Testing start job for scheduled job {scheduled_job_id}...")
    
    # Try to start the scheduled job
    start_response = client.api_call(f"api/v25.2/services/jobs/start_now/{scheduled_job_id}", method="POST")
    
    print(f"📤 Start job response: {start_response}")
    
    if start_response.get('responseStatus') == 'SUCCESS':
        print(f"✅ Successfully started job {scheduled_job_id}")
        
        # Wait a moment and check status
        print("⏱️ Waiting 2 seconds to check updated status...")
        time.sleep(2)
        
        status_check = client.api_call(f"api/v25.2/services/jobs/{scheduled_job_id}")
        if status_check.get('responseStatus') == 'SUCCESS':
            new_status = status_check.get('data', {}).get('status', 'Unknown')
            print(f"📊 Updated job status: {new_status}")
    else:
        print(f"❌ Failed to start job: {start_response}")
        error_type = start_response.get('errors', [{}])[0].get('type', 'Unknown')
        error_message = start_response.get('errors', [{}])[0].get('message', 'No message')
        print(f"🔍 Error type: {error_type}")
        print(f"💬 Error message: {error_message}")
else:
    print("⚠️ No scheduled job available for start testing")

🚀 Testing start job for scheduled job 264561...
📤 Start job response: {'responseStatus': 'SUCCESS', 'url': '/api/v25.2/services/jobs/264561', 'job_id': 264561}
✅ Successfully started job 264561
⏱️ Waiting 2 seconds to check updated status...
📊 Updated job status: QUEUEING
📊 Updated job status: QUEUEING


In [7]:
# Test: Try to create a simple job (document export as example)
if auth_success:
    print("🛠️ Testing job creation (document export)...")
    
    # First, let's check if we have any documents to export
    docs_response = client.api_call("api/v25.2/objects/documents?limit=1")
    
    if docs_response.get('responseStatus') == 'SUCCESS':
        documents = docs_response.get('data', [])
        if documents:
            doc_id = documents[0].get('id')
            print(f"📄 Found document for export testing: {doc_id}")
            
            # Try to export document (this should create a job)
            export_data = {
                'source': 'vault',
                'format': 'PDF'
            }
            
            export_response = client.api_call(
                f"api/v25.2/objects/documents/{doc_id}/file",
                method="POST",
                data=export_data
            )
            
            print(f"📤 Export response: {export_response}")
            
            if export_response.get('responseStatus') == 'SUCCESS':
                job_id = export_response.get('job_id')
                if job_id:
                    print(f"✅ Created export job: {job_id}")
                    print("🎯 This job can be used for further testing")
                else:
                    print("✅ Export successful but no job ID returned (synchronous operation)")
            else:
                print(f"❌ Export failed: {export_response}")
        else:
            print("⚠️ No documents found for export testing")
    else:
        print(f"❌ Could not check documents: {docs_response}")

🛠️ Testing job creation (document export)...
⚠️ No documents found for export testing
⚠️ No documents found for export testing


## 🎯 Jobs API Setup Analysis Summary

This notebook tested various aspects of the Jobs API:

1. **Job Histories** - ✅ Working (can retrieve completed jobs)
2. **Job Monitors** - ✅ Working (can retrieve active/scheduled jobs) 
3. **Individual Job Status** - ✅ Working (can get detailed job info)
4. **Job Execution Control** - ⚠️ Testing start_now functionality
5. **Job Creation** - ⚠️ Testing document export job creation

**Key Findings:**
- Jobs API core functionality is available and working
- The 404 errors in original testing were likely from specific job queue endpoints
- Job monitoring and status checking work reliably
- Some advanced job control features may require specific permissions

**Next Steps:**
- If setup confirms working functionality, re-run main Jobs API testing
- The 60% success rate suggests core APIs work but advanced features may be limited